# Multiple Dispatch and JIT

## The Julia Compiler

### 1. Julia uses JIT (just in time) compiler using LLVM for optimisation and code generation
### 2. LLVM - “Low Level Virtual Machine”. Now the full name of the project

![The Julia Compiler](./JIT.png)

## JIT Pipeline

![JITPipeline](./JITPipeline.png)


## Demo

In [1]:
f(x) = 2x^2+x+1

f (generic function with 1 method)

In [2]:
f(1)

4

In [3]:
f(1.0)

4.0

In [4]:
@code_native f(1)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movq	%rdi, %rax
	imulq	%rax, %rax
	leaq	1(%rdi,%rax,2), %rax
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [5]:
@code_native f(1.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movapd	%xmm0, %xmm1
	mulsd	%xmm1, %xmm1
	addsd	%xmm1, %xmm1
	addsd	%xmm0, %xmm1
	movabsq	$13172471248, %rax      ## imm = 0x31123F5D0
	addsd	(%rax), %xmm1
	movapd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopl	(%rax,%rax)


In [6]:
@code_llvm f(1)


define i64 @julia_f_71230(i64) #0 {
top:
  %1 = shl i64 %0, 1
  %2 = mul i64 %1, %0
  %3 = add i64 %0, 1
  %4 = add i64 %3, %2
  ret i64 %4
}


In [7]:
@code_llvm f(1.0)


define double @julia_f_71254(double) #0 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, 2.000000e+00
  %3 = fadd double %2, %0
  %4 = fadd double %3, 1.000000e+00
  ret double %4
}


In [8]:
@code_lowered f(1.0)

LambdaInfo template for f(x) at In[1]:1
:(begin 
        nothing
        return 2 * x ^ 2 + x + 1
    end)

In [9]:
methods(+)

# 163 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) at complex.jl:136
+(x::Bool, y::Bool) at bool.jl:48
+(x::Bool) at bool.jl:45
+{T<:AbstractFloat}(x::Bool, y::T) at bool.jl:55
+(x::Bool, z::Complex) at complex.jl:143
+(x::Bool, A::AbstractArray{Bool,N<:Any}) at arraymath.jl:91
+(x::Float32, y::Float32) at float.jl:239
+(x::Float64, y::Float64) at float.jl:240
+(z::Complex{Bool}, x::Bool) at complex.jl:137
+(z::Complex{Bool}, x::Real) at complex.jl:151
+(a::Float16, b::Float16) at float16.jl:136
+(x::Char, y::Integer) at char.jl:40
+(c::BigInt, x::BigFloat) at mpfr.jl:240
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:298
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:291
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:285
+(x::BigInt, y::BigInt) at gmp.jl:255
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:310
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:326
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:388
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:381
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:375
+(x::BigFloat, c::BigInt) at mpfr.jl:236
+(x::BigFloat, y::BigFloat) at mpfr.jl:205
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:212
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:220
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:228
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:38
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) at pkg/resolve/versionweight.jl:85
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) at pkg/resolve/versionweight.jl:131
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) at pkg/resolve/versionweight.jl:185
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) at dates/periods.jl:314
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) at dates/periods.jl:312
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) at dates/periods.jl:359
+(dt::DateTime, z::Base.Dates.Month) at dates/arithmetic.jl:37
+(dt::DateTime, y::Base.Dates.Year) at dates/arithmetic.jl:13
+(x::DateTime, y::Base.Dates.Period) at dates/arithmetic.jl:64
+(x::Date, y::Base.Dates.Day) at dates/arithmetic.jl:62
+(x::Date, y::Base.Dates.Week) at dates/arithmetic.jl:60
+(dt::Date, z::Base.Dates.Month) at dates/arithmetic.jl:43
+(dt::Date, y::Base.Dates.Year) at dates/arithmetic.jl:17
+(y::AbstractFloat, x::Bool) at bool.jl:57
+{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(x::T, y::T) at int.jl:32
+(x::Integer, y::Ptr) at pointer.jl:108
+(z::Complex, w::Complex) at complex.jl:125
+(z::Complex, x::Bool) at complex.jl:144
+(x::Real, z::Complex{Bool}) at complex.jl:150
+(x::Real, z::Complex) at complex.jl:162
+(z::Complex, x::Real) at complex.jl:163
+(x::Rational, y::Rational) at rational.jl:199
+(x::Integer, y::Char) at char.jl:41
+{N}(i::Integer, index::CartesianIndex{N}) at multidimensional.jl:58
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:314
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:327
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:216
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:224
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:232
+(x::Irrational, y::Irrational) at irrationals.jl:88
+(x::Number) at operators.jl:115
+{T<:Number}(x::T, y::T) at promotion.jl:255
+(x::Number, y::Number) at promotion.jl:190
+(r1::OrdinalRange, r2::OrdinalRange) at operators.jl:505
+{T<:AbstractFloat}(r1::FloatRange{T}, r2::FloatRange{T}) at operators.jl:512
+{T<:AbstractFloat}(r1::LinSpace{T}, r2::LinSpace{T}) at operators.jl:531
+(r1::Union{FloatRange,LinSpa

In [10]:
p(x::Int64) = 2x
p(x::Float64) = 3x
p(x,y) = x+y
p(x::AbstractString) = "hello world"

p (generic function with 4 methods)